In [1]:
from pygsti.objects import GateString
from pygsti.construction import list_all_gatestrings_onelen
import itertools as _itertools

In [2]:
def itertools_onelen(gate_labels, length):
    gate_tuples = _itertools.product(gate_labels, repeat=length)
    return list(map(GateString, gate_tuples))

In [3]:
myGates = [ 'Gx', 'Gy' ]

## Performance

Check to see which implementation performs faster.

### Long sequences

In [4]:
%timeit -n 10 list_all_gatestrings_onelen(myGates, 15)

10 loops, best of 3: 190 ms per loop


In [5]:
%timeit -n 10 itertools_onelen(myGates, 15)

10 loops, best of 3: 43.9 ms per loop


### Short sequences

In [6]:
%timeit -n 10 list_all_gatestrings_onelen(myGates, 5)

10 loops, best of 3: 210 µs per loop


In [7]:
%timeit -n 10 itertools_onelen(myGates, 5)

10 loops, best of 3: 45.8 µs per loop


## Correctness

Make sure the new implementation gives the same results.

In [8]:
all([list_all_gatestrings_onelen(myGates, length) == itertools_onelen(myGates, length)
     for length in range(15)])

True

The implementation using itertools seems to perform correctly and about 4x as quickly
as the current implementation.